In [7]:
#Imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
from scipy.stats import pointbiserialr
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm

import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [8]:
#connection
conn = snowflake.connector.Connect(user='ANALYST', password='Fedewap2023!', account='rl43903.east-us-.azure')
sql = "select * from chicagofire_database.salesforce.sf_formstacks where FORM_NAME__C in ('2024 Renewal Member Survey', '2024 Non-Renewal Member Survey')"
res = conn.cursor().execute(sql).fetchall()
#load in dataset
df = pd.read_sql(sql, conn)

In [9]:
df.columns.to_list()

['ATTRIBUTES',
 'ID',
 'NUMBER_OF_TICKETS__C',
 'AD_CAMPAIGN_NAME__C',
 'AD_NAME__C',
 'AD_PLATFORM__C',
 'AD_SET_COLLECTED_DATE__C',
 'AD_SET_NAME__C',
 'AGE_GROUP__C',
 'ALLSTATE_OPT_IN__C',
 'BURNHAM_COMMENTS__C',
 'BURNHAM_DOWNS__C',
 'BURNHAM_ENTERTAINMENT_RATING__C',
 'BURNHAM_FOOD_RATING__C',
 'BURNHAM_MUSIC_RATING__C',
 'STAFF_RATING__C',
 'CASHIER_EXPERIENCE__C',
 'CFFC_CLASS_RATING__C',
 'CFFC_SCHEDULE_MULTI_SELECT__C',
 'CFRS_ADD_ON__C',
 'CHECKBOX_1__C',
 'CHECKBOX_2__C',
 'CHECKBOX_3__C',
 'CHECKBOX_4__C',
 'CHILD_SOCCER_ATTITUDE__C',
 'COACHING_QUALITY_RATING__C',
 'COMMENTS__C',
 'COMMUNICATION_RATING__C',
 'COMPETITION_QUALITY_RATING__C',
 'CONCESSIONS__C',
 'CONCESSIONS_COMMENTS__C',
 'CONCESSIONS_FOOD_QUALITY_RATING__C',
 'CONCESSIONS_VALUE_RATING__C',
 'CONCESSIONS_VARIETY_RATING__C',
 'CONCESSIONS_WAIT_TIME_RATING__C',
 'CONTACT__C',
 'COVID_19_SAFETY_RATING__C',
 'CREATEDBYID',
 'CURRENCY_1__C',
 'CURRENCY_2__C',
 'CURRENT_CLUB__C',
 'DATE_1__C',
 'DATE_2__C',
 'DA

Need to make a subdf of just the columns that are open ended q's - noting what q they are in the column name. From there, for each column can perform thematic analysis.

Renewal Survey:

First Name  FirstName
Last Name   LastName
Email   Email
How many years have you been a STM - TEXT_1__C
How do you use tickets - SINGLE_VALUE_PICKLIST_1__C
Who is your rep? - SINGLE_VALUE_PICKLIST_2__C
Friendliness    Contact.Formstack_Survey__c.A.Friendliness_Rating__c	
Response Time   Contact.Formstack_Survey__c.A.Response_Time_Rating__c	
Problem Solving Ability	    Contact.Formstack_Survey__c.A.Problem_Solving_Rating__c
Attentiveness	Contact.Formstack_Survey__c.A.Rep_Attentiveness__c	
Patience	Contact.Formstack_Survey__c.A.Patience_Rating__c		
Communication	Contact.Formstack_Survey__c.A.Rep_Communication_Rating__c		
Product Knowledge	Contact.Formstack_Survey__c.A.Product_Knowledge_Rating__c		
Professionalism	Contact.Formstack_Survey__c.A.Professionalism__c		
What benefits associated with your membership do you value most? (please select )	Contact.Formstack_Survey__c.A.Multi_Value_Picklist_1__c		
What benefits do you utilize most? (please select )	Contact.Formstack_Survey__c.A.Multi_Value_Picklist_2__c		
How do you usually get to games?	Contact.Formstack_Survey__c.A.Single_Value_Picklist_4__c		
Do you regularly purchase food and/or beverages from the concessions?	Contact.Formstack_Survey__c.A.Concessions__c		
Did you purchase merchandise?	Contact.Formstack_Survey__c.A.Merchandise__c		
Parking Lot Arrival Time	Contact.Formstack_Survey__c.A.Parking_Lot_Arrival_Time__c		
Parking Lot Departure Time	Contact.Formstack_Survey__c.A.Parking_Lot_Departure_Time__c		
Staff (were the staff helpful and friendly?)	Contact.Formstack_Survey__c.A.Number_1__c		
Parking Value	Contact.Formstack_Survey__c.A.Number_2__c		
Any additional comments on parking?	Contact.Formstack_Survey__c.A.Parking_Comments__c		
How would you rate your experience entering the stadium?	Contact.Formstack_Survey__c.A.Stadium_Gate_Rating__c		
Any additional comments about your experience entering the stadium?	Contact.Formstack_Survey__c.A.Comments__c		
Overall Wait Time ("Very Satisfactory" being little to no wait time)	Contact.Formstack_Survey__c.A.Concessions_Wait_Time_Rating__c		
Food/Beverage Variety	Contact.Formstack_Survey__c.A.Concessions_Variety_Rating__c		
Concessions Staff (Were they helpful and friendly?)	Contact.Formstack_Survey__c.A.Number_3__c		
Food/Beverage Value	Contact.Formstack_Survey__c.A.Concessions_Value_Rating__c		
Any additional comments on concessions services?	Contact.Formstack_Survey__c.A.Concessions_Comments__c		
Overall Wait Time ("Very Satisfactory" being little to no wait time)	Contact.Formstack_Survey__c.A.Merchchandise_Wait_Time__c		
Merchandise Variety	Contact.Formstack_Survey__c.A.Merchchandise_Variety__c		
Merchandise Value	Contact.Formstack_Survey__c.A.Merchandise_Value_Rating__c		
Merchandise Quality	Contact.Formstack_Survey__c.A.Merchandise_rating__c		
Merchandise Staff (were they helpful and friendly?	Contact.Formstack_Survey__c.A.Number_4__c		
Any additional comments on merchandise?	Contact.Formstack_Survey__c.A.Merchandise_Comments__c		
New 2024 Member Benefits	Contact.Formstack_Survey__c.A.New_Benefits_Rating__c
Family Friendly Atmosphere	Contact.Formstack_Survey__c.A.Family_friendly_rating__c
Sense of Community	Contact.Formstack_Survey__c.A.Sense_of_Community__c
What was the main reason you renewed your membership for the 2024 season?   Contact.Formstack_Survey__c.A.Long_Text_1__c		
How likely are you to recommend being a Chicago Fire FC Member to a friend or colleague Contact.Formstack_Survey__c.A.NPS__c		
Are you interested in exploring any of our Fieldside, Skyline, or Suite opportunities?  Contact.Formstack_Survey__c.A.Single_Value_Picklist_5__c
Please provide any additional feedback to help us improve your Season Ticket Member experience. Contact.Formstack_Survey__c.A.Long_Text_2__c	